In [1]:
from datasets import load_dataset

dataset = load_dataset('THUDM/LongBench', 'qasper', trust_remote_code=True)

/opt/anaconda3/envs/hmt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    test: Dataset({
        features: ['input', 'context', 'answers', 'length', 'dataset', 'language', 'all_classes', '_id'],
        num_rows: 200
    })
})

In [3]:
import datasets

def prepare_qasper(dataset: datasets.Dataset):
    prompt = "Prompt: Answer the question based on the provided scientific context."
    question_delimiter = "Question: "
    context_delimiter = "Scientific Context: "
    answer_delimiter = "Answer: "

    def qasper_entry_to_text_longbench(dataset):
        """
        Convert a QMSum dataset entry to a text entry. 
        The text is a concat of prompt, question delimiter, question, context delimiter, context and label is the answer.
        """
        return {
            "text": [f"{prompt}\n{question_delimiter}{entry[0]}\n{context_delimiter}{entry[1]}\n{answer_delimiter}{entry[2][0]}" for entry in zip(dataset['input'], dataset['context'], dataset['answers'])],
            "answer": [entry[0] for entry in dataset['answers']],
        }

    dataset = dataset.map(qasper_entry_to_text_longbench, batched=True, 
                                                    desc=f"Concatenating QA Input Questions and Contexts",
                                                    num_proc=8).remove_columns(['input', 'context', 'answers', 'length', 
                                                                                'dataset', 'language', 'all_classes', '_id'])
    return dataset


In [4]:
from tools.data_processing.generic import prepare_test
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("facebook/opt-350m")

ds = prepare_test(dataset, prepare_qasper, max_token_num=100000, test_length=1000000, block_size=1024, tokenizer=tok, with_answer=True)

DatasetDict({
    test: Dataset({
        features: ['text', 'answer'],
        num_rows: 200
    })
})
DatasetDict({
    test: Dataset({
        features: ['text', 'answer', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})


/opt/anaconda3/envs/hmt/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyError: 'answer'

# Test ArgParser


In [4]:
from argparse import ArgumentParser

parser = ArgumentParser()
args = parser.parse_args(args=[])

usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: --max-depth=0


SystemExit: 2

/opt/anaconda3/envs/hmt/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
args


Namespace()